In [614]:
##importing the Iris data and few modules
import math
import numpy as np
import pandas as pd
from sklearn import datasets


## loading the dataset
iris = datasets.load_iris()

# making training data 
X_train = iris['data']
# making testing data
Y_train = iris['target']

## converting into dataframe
X_train = pd.DataFrame(X_train)

Y_train = pd.DataFrame(Y_train)

# assigning columns names for easier referene
X_train.columns = iris['feature_names']
Y_train.columns = ['class']

# creatinf features list for easier removal of feature after splitting on it
features = list(X_train.columns)




In [615]:
# function to calculate entropy for us , the formula is implemented here 

def entropy(Y):
    # number of samples belonging to class 0 , 1 , 2 are found respectively for calculating probability
    a = len(Y[Y['class']==0])
    b = len(Y[Y['class']==1])
    c = len(Y[Y['class'] == 2])
    entropy = 0 
    # list made of these same values for  iterative ease 
    lis = [len(Y[Y['class']==0]),len(Y[Y['class']==1]),len(Y[Y['class'] == 2])]
    for i in lis:
        if i == 0:
            entropy+=0
            continue
            #Entropy formula is implemented here for all the three clases , if a class dosent exisit zero is added 
        entropy += -((i/(a+b+c))*(math.log((i/(a+b+c)),10)))
    return entropy

# the below functions helps to find the gain it takes training data , target , feature to split on and the current entropy

# simple formula calculation is implemented for each unique value of the X(training data) for a particular feature and we 
# will obviously select the feature and the value for which we get maximum gain in information , the same has been implemented
   
def gain(X,Y,feature,curr_entropy):
    # unique values in a feature column for training data is realised
    X_vals = set(X[feature])
    en_val = 1000000
    feat = ''
    valu = 0
    entro = 0
    # finding minimum entropy for a value so we can get maximum gain 
    for val in X_vals:
        entro = 0
        index1 = pd.Index(X[feature]<val)
        index2 = pd.Index(X[feature]>=val)
        Y_for_that_val = Y[index1]
        Y_not_for_that_val = Y[index2]
        
        # (len(Y[index?])/len(Y)) ==> this term is nothing but the weighted avergae for each entropy term
        
        entro = (len(Y[index1])/len(Y)) * entropy(Y_for_that_val) + (len(Y[index2])/len(Y)) * entropy(Y_not_for_that_val)
        
        # below if statements helps pick the minimum value of entropy 
        
        if entro<en_val:
            en_val = entro
            valu = val
    # gain is the difference of parent entropy - weighted average * child's entropy 
    # gain is calculated and returned along with value in feature for which highest gain is possible in split
    gain = curr_entropy - en_val
    return gain , valu
    
## this is the function to help call the recursion for children nodes

def dt(X,Y,features,level):
    
    # if our targets are just of one type then we have encountered a pure node 
    if len(set(Y.iloc[:,0]))==1:
        print('Level ' , level)
        ans = -3
        for x in set(Y.iloc[:,0]):
            ans = x
        print('Count of ',ans," is = ",len(Y))
        print('Current Entropy is = 0.0')
        print('Reached leaf Node')
        print()
        return 
        
    # if there is no target then we should return because there is no data left 
    
    if(len(set(Y.iloc[:,0]))==0):
        return
    
    # this is for when we have run out of features to split on , here a majority vote is taken on the basis of target values
    if(len(features)==0):
        print('Level ' , level)
        #finding the maximum number of a particular type of class present in this node
        max_vals = max(Y['class'].value_counts())
        count_val = -2
        for i in range(3):
            try:
                # finding out the class having maximum presence in the node 
                if Y['class'].value_counts()[i] == max_vals:
                    count_val = i
            except:
                continue      
        print('Count of ',count_val," is = ",max_vals)
        print('Majority vote is taken here')
        print('Current Entropy is = ', entropy(Y))
        print('Reached leaf Node')
        print()
        return
    
    # If none of the above conditions occured then we are safe to proceed with the binary split because we have features
    #  , data and a varied range of class values in our target so more splitting needs to be done
    
    # basic prints required for output 
    
    print('Level ',level)
    try:
        count_0 = Y.iloc[:,0].value_counts().loc[0]
    except:
        count_0 = 0
    print('Count of 0 = ',count_0)
    try:
        count_1 = Y.iloc[:,0].value_counts().loc[1]
    except:
        count_1 = 0
    print('Count of 1 = ',count_1)
    try:
        count_2 = Y.iloc[:,0].value_counts().loc[2]
    except:
        count_2 = 0
    print('Count of 2 = ',count_2)
    
    
    
    # here we proceed to find the best feature to split on and the best value to split on in that feature
    # we will simply pass all features and let the gain function do its job
    # the feature with max gain will be selected
    
    max_gain = -100000
    feature_to_split_on = ''
    value_to_split_on = 0
    
    # initial entropy was found here
    
    current_entropy = entropy(Y)
    
    #calling gain on all features to find best feature to split on and the value
    for f in features:
        gain_of_splitting_on_f , value_to_split_on_1= gain(X,Y,f,current_entropy)
        if(gain_of_splitting_on_f>max_gain):
            max_gain = gain_of_splitting_on_f                     ## max gain stored here
            feature_to_split_on = f                               ## feature name stored here
            value_to_split_on = value_to_split_on_1               ## value to split on stored here
    #Divide data for each of the two nodes one which is lesser than the value to split on for that feature
    #One which is greater than or equal to the value to split on for that feature
    
    index1 = pd.Index(X[feature_to_split_on]<value_to_split_on)
    index2 = pd.Index(X[feature_to_split_on]>=value_to_split_on)
                      
    # basic few prints for output 
    
    print('Current entropy is ',current_entropy)
    print('Splitting on feature ',feature_to_split_on,' with gain ratio ',max_gain)
    print('Value to split on ',value_to_split_on)
    print()
   
    # Since we already divided data for the feature to split on lets remove it from the features list so its not used again
    features.remove(feature_to_split_on)
    
    ## Dividing training data and targets according to value mentioned above for the binary split
                      
    X_train_1 = X[X[feature_to_split_on]<value_to_split_on]
    Y_train_1 = Y[index1]
                      
    # Calling recursion for the first child and also incrementing its level because it will be a child
    dt(X_train_1,Y_train_1,features,level+1)
                      
    # Dividing data for the second child
                      
    X_train_2 = X[X[feature_to_split_on]>=value_to_split_on]
    Y_train_2 = Y[index2]
    # calling recursion for the second child 
    dt(X_train_2 , Y_train_2, features,level+1)

In [616]:
# Calling function to print output

dt(X_train,Y_train,features,0)


Level  0
Count of 0 =  50
Count of 1 =  50
Count of 2 =  50
Current entropy is  0.4771212547196624
Splitting on feature  petal length (cm)  with gain ratio  0.27643459094367495
Value to split on  3.0

Level  1
Count of  0  is =  50
Current Entropy is = 0.0
Reached leaf Node

Level  1
Count of 0 =  0
Count of 1 =  50
Count of 2 =  50
Current entropy is  0.30102999566398114
Splitting on feature  petal width (cm)  with gain ratio  0.20775897341573138
Value to split on  1.8

Level  2
Count of 0 =  0
Count of 1 =  49
Count of 2 =  5
Current entropy is  0.13397787198820432
Splitting on feature  sepal length (cm)  with gain ratio  0.019926516393152488
Value to split on  7.2

Level  3
Count of 0 =  0
Count of 1 =  49
Count of 2 =  4
Current entropy is  0.11620326796476979
Splitting on feature  sepal width (cm)  with gain ratio  0.018521175299525544
Value to split on  2.9

Level  4
Count of  1  is =  27
Majority vote is taken here
Current Entropy is =  0.16700486810509502
Reached leaf Node

Lev